In [27]:
import re 
import spacy  
from spacy import displacy 
import en_core_web_sm 
from pathlib import Path 
from IPython.display import display, Markdown, Latex
import sys 
import csv 
import pandas as pd 
import numpy as np 
import os 
from os import path 
from datetime import datetime 
from urllib.parse import urlparse, urljoin 
import requests 
import ntpath 
from bs4 import BeautifulSoup, Comment 
import time 
from pprint import pprint  
import shutil 
import tldextract 
import multiprocessing 
from multiprocessing import Process, Queue, cpu_count 
import os 
from os import path, listdir
###
from IPython.core.debugger import set_trace
import random
from fractions import Fraction

In [28]:
from IPython.display import display, Markdown, Image
display(Markdown(
"""
# Midterm Project Learning Status Check
## <font color="red">Due: Feb 1, 2020</font>
## Open Book Midterm:
<ol>
    <li><b>(200 Points)Project Statement</b><br>: What are we doing? and in high level, how we are doing it <br>You may review Phase I/II PowerPoints.<br>Words limit: 500 
    <br></li>
    <br><li><b>(200 Points)Impersonate and Search</b><br>:Sign in US News Compass and conduct 3 to 5 searches. <br> Save the results to HTML files (one file per search) 
    <br><a href="https://premium.usnews.com/best-colleges/myfit?gpa=4.0&mode=table&sat-composite=1580">Compass Search</a>
    <br>A sample file from search may look like this: <br>
    <a href='./img/OneSampleCompassHtml.html'> One Sample file</a>
    </li>
    <br><li><b>(800 Points)Python and Content</b><br>Design and Implement a Class to extrat result from pages generated 
    from previous question. <br>Ideal output format should reflect at the following information:  </li>
    <br> Search Criteria: <br><img src="./img/CompassSearchFields.PNG" width="500"/>
    <br> Search Results: <br><img src="./img/CompassSearchResults.PNG" width="500"/></li>
    <br><li><b>(400 Points)Scratching NLTK</b><br>Crawl the following page<br>
    <a href="https://www.nme.com/features/greta-thunberg-full-speech-to-the-un-2019-climate-change-summit-2550824">Greta Thunberg UN Speech</a></li>
    <br> Tasks: 
    <br><ul>
        <li>Get the content of her speech</li>
        <li>Build a dataset that list all words in this speech and how many times each word appears</li>
        <li>Sort the list of words by <br>a).Alphabetically. <br> b). by frequency
        <li>Lemmatize all words</li>
        <li>Choose ~30 "difficult" words and get the synonyms and antoyms for them.</li> 
    </ul>
    </li>
</ol>
"""
))   


# Midterm Project Learning Status Check
## <font color="red">Due: Feb 1, 2020</font>
## Open Book Midterm:
<ol>
    <li><b>(200 Points)Project Statement</b><br>: What are we doing? and in high level, how we are doing it <br>You may review Phase I/II PowerPoints.<br>Words limit: 500 
    <br></li>
    <br><li><b>(200 Points)Impersonate and Search</b><br>:Sign in US News Compass and conduct 3 to 5 searches. <br> Save the results to HTML files (one file per search) 
    <br><a href="https://premium.usnews.com/best-colleges/myfit?gpa=4.0&mode=table&sat-composite=1580">Compass Search</a>
    <br>A sample file from search may look like this: <br>
    <a href='./img/OneSampleCompassHtml.html'> One Sample file</a>
    </li>
    <br><li><b>(800 Points)Python and Content</b><br>Design and Implement a Class to extrat result from pages generated 
    from previous question. <br>Ideal output format should reflect at the following information:  </li>
    <br> Search Criteria: <br><img src="./img/CompassSearchFields.PNG" width="500"/>
    <br> Search Results: <br><img src="./img/CompassSearchResults.PNG" width="500"/></li>
    <br><li><b>(400 Points)Scratching NLTK</b><br>Crawl the following page<br>
    <a href="https://www.nme.com/features/greta-thunberg-full-speech-to-the-un-2019-climate-change-summit-2550824">Greta Thunberg UN Speech</a></li>
    <br> Tasks: 
    <br><ul>
        <li>Get the content of her speech</li>
        <li>Build a dataset that list all words in this speech and how many times each word appears</li>
        <li>Sort the list of words by <br>a).Alphabetically. <br> b). by frequency
        <li>Lemmatize all words</li>
        <li>Choose ~30 "difficult" words and get the synonyms and antoyms for them.</li> 
    </ul>
    </li>
</ol>


In [29]:
#1. What we are doing is to find the best college for a person based on their info. How we can do this is by using python, react, web crawling, and AI to produce an accurate output.

In [ ]:
#3

In [ ]:
#4

In [4]:
display(Markdown(
"""
# Python Exam (For Python students):
<ol>
<li>(100 points)Write a program that contains a while loop that repeatedly asks the user to enter numeric miles they run each week.  Append each run to a list called “distance”. You may use this format [(“week1”, 23.1), (“week2”, 12.2)…]  
</li><br><li>(100 points)Write a program that get the max, min, average, number of runs etc. 
</li><br><li>(100 points)Write a program that prompts the user to enter in an unlimited number of colors. Test to make sure that the user did not already enter a color before storing it (i.e. “We’re sorry, but you’ve already added the color ‘yellow’”). When the user types the string “exit” you should stop collecting colors. The print out the colors entered in alphabetical order.  
</li><br><li>(100 points)Write a function that generates a random letter from the string ‘BINGO’. Here is a sample running of this program: x = generate_bingo_letter() print (x) 
</li><br><li>(100 points)Define a function fraction(fstring), where the argument passed in stores a fraction as a string. The function should return a floating point number that is the value of the fraction. 
 
Examples of using the function: <br>
         print(fraction(“3/4”))  # prints  .750 <br>
         print(fraction(“1/3”))  # prints   0.333 <br>
     
 
</li><br><li>(Bonus. 300 Points) Write a function balance_parens(astring) that inputs a string with parenthesized sub-­‐strings, which may be nested (e.g., could be a Python arithmetic expression). The function returns True if all of the parentheses that are opened are properly closed, otherwise it returns False. 
<br>
For example, the following string is balanced: 
<br>
“ A * ( (B + C) – (D – E) )” 
<br>
And the following string is not balanced: 
<br> 
“  A ((B * C)  + (D      
</li>
</ol>

"""
))


# Python Exam (For Python students):
<ol>
<li>(100 points)Write a program that contains a while loop that repeatedly asks the user to enter numeric miles they run each week.  Append each run to a list called “distance”. You may use this format [(“week1”, 23.1), (“week2”, 12.2)…]  
</li><br><li>(100 points)Write a program that get the max, min, average, number of runs etc. 
</li><br><li>(100 points)Write a program that prompts the user to enter in an unlimited number of colors. Test to make sure that the user did not already enter a color before storing it (i.e. “We’re sorry, but you’ve already added the color ‘yellow’”). When the user types the string “exit” you should stop collecting colors. The print out the colors entered in alphabetical order.  
</li><br><li>(100 points)Write a function that generates a random letter from the string ‘BINGO’. Here is a sample running of this program: x = generate_bingo_letter() print (x) 
</li><br><li>(100 points)Define a function fraction(fstring), where the argument passed in stores a fraction as a string. The function should return a floating point number that is the value of the fraction. 
 
Examples of using the function: <br>
         print(fraction(“3/4”))  # prints  .750 <br>
         print(fraction(“1/3”))  # prints   0.333 <br>
     
 
</li><br><li>(Bonus. 300 Points) Write a function balance_parens(astring) that inputs a string with parenthesized sub-­‐strings, which may be nested (e.g., could be a Python arithmetic expression). The function returns True if all of the parentheses that are opened are properly closed, otherwise it returns False. 
<br>
For example, the following string is balanced: 
<br>
“ A * ( (B + C) – (D – E) )” 
<br>
And the following string is not balanced: 
<br> 
“  A ((B * C)  + (D      
</li>
</ol>



In [30]:
display(Markdown(
"""
## You may use the following libraries for the exams. 
import re <br>
import spacy  <br>
from spacy import displacy <br>
import en_core_web_sm <br>
from pathlib import Path <br>
from IPython.display import display, Markdown, Latex  <br>
import sys <br>
import csv <br>
import pandas as pd <br>
import numpy as np <br>
import os <br>
from os import path <br>
from datetime import datetime <br>
from urllib.parse import urlparse, urljoin <br>
import requests <br>
import ntpath <br>
from bs4 import BeautifulSoup, Comment <br>
import time <br>
from pprint import pprint  <br>
import shutil <br>
import tldextract <br>
import multiprocessing <br>
from multiprocessing import Process, Queue, cpu_count <br>
import os <br>
from os import path, listdir  <br>
"""
))



## You may use the following libraries for the exams. 
import re <br>
import spacy  <br>
from spacy import displacy <br>
import en_core_web_sm <br>
from pathlib import Path <br>
from IPython.display import display, Markdown, Latex  <br>
import sys <br>
import csv <br>
import pandas as pd <br>
import numpy as np <br>
import os <br>
from os import path <br>
from datetime import datetime <br>
from urllib.parse import urlparse, urljoin <br>
import requests <br>
import ntpath <br>
from bs4 import BeautifulSoup, Comment <br>
import time <br>
from pprint import pprint  <br>
import shutil <br>
import tldextract <br>
import multiprocessing <br>
from multiprocessing import Process, Queue, cpu_count <br>
import os <br>
from os import path, listdir  <br>


In [31]:
#1
def run(weeks=10):
    distance = []
    week = 1
    while week <= weeks:
        try:
            miles = input('How many miles have you run in week' + str(week) + '\n')
            milesint = float(miles)
            pair = ('week' + str(week), milesint)
            week += 1
            distance.append(pair)
        except ValueError:
            print("Please enter a number")
            continue

            
    return(distance)
run()

How many miles have you run in week1
1
How many miles have you run in week2
2
How many miles have you run in week3
3
How many miles have you run in week4
4
How many miles have you run in week5
5
How many miles have you run in week6
6
How many miles have you run in week7
7
How many miles have you run in week8
8
How many miles have you run in week9
9
How many miles have you run in week10
10


[('week1', 1.0),
 ('week2', 2.0),
 ('week3', 3.0),
 ('week4', 4.0),
 ('week5', 5.0),
 ('week6', 6.0),
 ('week7', 7.0),
 ('week8', 8.0),
 ('week9', 9.0),
 ('week10', 10.0)]

In [64]:
#2
y=0
data= [('week1', 23.0),
 ('week2', 4334.0),
 ('week3', 324.0),
 ('week4', 4.0),
 ('week5', 35321.0),
 ('week6', 562134.0),
 ('week7', 43214.0),
 ('week8', 5.0),
 ('week9', 2354.0),
 ('week10', 5.0)]

number_of_runs = len(data)

max_miles=max(data, key=lambda x: x[1])
min_miles=min(data, key=lambda x: x[1])
avg=sum(x[1] for x in data)/number_of_runs

print("The farthest distance you've run is " + str(max_miles[1]) + ' miles ' + 'on ' + str(max_miles[0]))
print("The shortest distance you've run is " + str(min_miles[1]) + ' miles ' + 'on ' + str(min_miles[0]))
print("Your average distance ran is " + str(avg) + " miles for " + number_of_runs)
print("You've gone on " + number_of_runs + " runs so far.")

        



The farthest distance you've run is 562134.0 miles on week6
The shortest distance you've run is 4.0 miles on week4


TypeError: can only concatenate str (not "int") to str

In [33]:
#3
def listcolors():
    colors = []
    while True:
        usercolor = input("Add a color: ")
        if usercolor == "exit":
            break
            
        if usercolor in colors:
            print("We’re sorry, but you’ve already added the color " + usercolor)
            continue
        else:
            colors.append(usercolor)
        
    print(sorted(colors))
            
listcolors()

Add a color: dsa
Add a color: dsa
We’re sorry, but you’ve already added the color dsa
Add a color: dsad
Add a color: das
Add a color: dsa
We’re sorry, but you’ve already added the color dsa
Add a color: ddsa
Add a color: few
Add a color: 324
Add a color: 3214
Add a color: 6ret
Add a color: trfdsf
Add a color: exit
['3214', '324', '6ret', 'das', 'ddsa', 'dsa', 'dsad', 'few', 'trfdsf']


In [37]:
#4
def generate_bingo_letter():
    bingo="BINGO"
    
    rand = random.randint(0,4)        
    return bingo[rand]
    
    #return random.choice(bingo)


x = generate_bingo_letter()
print(x) 

O


In [38]:
#5
def fraction(fstring):
    sum = 0
    splitfract=fstring.split()
    if len(splitfract) != 1 and len(splitfract) != 2:
        print("invalid fraction")
        sys.exit()
        
       
    for x in splitfract:
        try:
            a = Fraction(x)
            rounded = round(float(a),3)
            sum = sum + rounded
        except ValueError as e :
            print(e)
    return(sum)


fraction('74398231 3841234/1324132')

74398233.901

In [43]:
#6
def balance_parens(astring):
    thestack = []
    index = 0
    
    
    while index < len(astring):
        lpar = astring[index]
        if lpar == "(":
            thestack.append(lpar)
        elif lpar == ')':
            if len(thestack) == 0:
                return False
            else:
                thestack.pop()
        index = index + 1
        
        
    if len(thestack) == 0:
        return True
    else:
        return False


balance_parens("(((((kdkdksadjiajsd))))")
        

False

NameError: name 'mean' is not defined